In [6]:
import urllib
import json
import pandas as pd
import numpy as np
import warnings
import pickle
from datetime import datetime
from datetime import timedelta
import statistics
warnings.filterwarnings('ignore')

In [101]:
PICKLE_PATH = 'hopkins_conf_augmented0304.pkl'
RESULT_PATH = 'hopkins.conf.withgrowh.pkl'

###### Open pickle file

In [96]:


hopkins_confirmed = pd.DataFrame()
 
hpcon_rates=pd.DataFrame()

with open(PICKLE_PATH, 'rb') as file:
    hopkins_confirmed = pickle.load(file)
    
hopkins_confirmed
hopkins_confirmed.insert(7,column='last relevant date',value=' ')
    
        

#hpcon_datarows = pd.DataFrame()
#hpcon_datarows =hopkins_confirmed.index
    
#for row in hopkins_confirmed.iterrows():
#     hpcon_datarows.append(row)

#hpcon_datarows

In [122]:
datarows = [row for row in hopkins_confirmed.iterrows()  if row[0][1]=='data']
#hopkins_confirmed.insert(7,column='last relevant date',value=np.nan)

 


### Obtain a list of dates (string of columns names which are dates , assumed to be a continouos sequence in the DF)


cur=0

for name in hopkins_confirmed.columns :
    if '2020' in name and '/' in name: # this is a date!
        datefound=True
        break
    cur +=1  
if datefound == False:
    print ('date not found in DF columns')
else:
    print ('first date is',name, 'offset',cur )
    firstdate=name
    firstdate_offset=cur 
    
rvrscur=len(hopkins_confirmed.columns)

for name in hopkins_confirmed.columns[::-1]:                #this loop will allow columns which are not dates right of dates
    if '2020' in name and '/' in name: # this is a date!
        break # this is  a date...
    else:
        rvrscur -=1
print ('rvrscur col',hopkins_confirmed.columns[rvrscur-1])
valid_dates=hopkins_confirmed.columns[cur:rvrscur]         # valid dates - all the dates in the index

print ('dates',valid_dates)
 

for row in datarows:
    print (row[0],row[0][1])
    ratios=[]
    totals=[]
    actual_dates=[]
    
    for cursor in range (len(valid_dates)-1):
        p=row[1][valid_dates[cursor]]
        q=row[1][valid_dates[cursor+1]  ]
        actual_dates.append(valid_dates[cursor])
        if p > 0:
            ratios.append (round(q/p - 1,4 ))
            totals.append (p)
        else:
            if q>0:
                ratios.append (4)           # the first gr is set to 4 (5 times ...) and will most often get removed due to
                                            # totals less then 5 pct
            else: ratios.append(0)          #  0 growth between two zeros...
            totals.append(0)
            
    ratios=[0]+ratios  # so that each growth rate 'sits' under its 'total' and is the growth between this and the 
                       # previous
    totals.append(row[1][valid_dates[-1]]) # add the last total to list
           
    print ('==========')        
    print  ('new',row[1]['Province_State'],row[1][ 'Country_Region'])
    print ('totals',totals)
    print ('ratios',ratios)
       
    if max(totals)<20:
        print ('SKIP')
        continue    ###  if no day with 20 or more, skip this region,  small numbers 
    print ('NO SKIP')
        
   
    print (row[1]['Province_State'],row[1][ 'Country_Region'],ratios,totals)
    print ('lens',len(ratios),len(totals))
    maxt=max(totals)
    few=maxt/20
    print (row[1]['Province_State'],row[1][ 'Country_Region'],maxt,few)
    # here , find the 5% seperator
    few_seperator = -1
    for cursor in range (len(totals)-1):
        if  (totals[cursor] <=few ) and (totals [cursor+1]>=few):
            few_seperator=cursor
            break
    if few_seperator >=0 :
        t_sample=totals[cursor+1:]
        t_ratios=ratios[cursor+1:]
    if cursor < len(totals)-2   :
        five_pct_name=actual_dates[cursor+1] # ths first date where more than 5% of max
        actual_dates=actual_dates[cursor+1:]
        print ('five pct', five_pct_name) 
    else:
        print ('five pct not found, zeros')
 
    #   find if there is a sequence of growth rate lower than 2% per day (from the end )  remove it if found
    
    for cur in range (len(t_ratios)):
        maxright=max(t_ratios[cur:])
        if   (maxright>0.02):
            continue
        else:
            print ('maxright ,cur, len ratios, rightside',maxright,cur,len(t_ratios),t_ratios[cur:],t_sample[cur:] )
            break
    
    
    last_relevant_sample=actual_dates[ -1]  # overriden if cut   
    if cur < len(t_ratios)-1:
     
        print ('cut right side len befor , len after',len(t_ratios),cur+1)
        
        t_ratios=t_ratios [:cur+1]
        t_sample=t_sample [:cur+1]
        actual_dates=actual_dates[:cur+1]
        last_relevant_sample=actual_dates[ -1]
        
    else: print ('nothing was cut')
            
    
    sorted_sample=t_sample.sort()
    
    
    print ('sample size',len(t_sample),t_sample,t_ratios)
    
    med=statistics.median(t_ratios)
    
    lower=np.percentile(t_ratios,25)
    upper=np.percentile(t_ratios,75)
    
    print ('lower, median, upper',lower,med,upper)
    
    colslen=len(hopkins_confirmed)
     
    hopkins_confirmed.loc[(row[0][0],'Growth rates'),:]=   (  pd.Series([np.nan]*colslen) )     # add the row
    
    hopkins_confirmed.at[(row[0][0],'Growth rates'),'5%_Date']=five_pct_name
    print ('last',last_relevant_sample)
    hopkins_confirmed.at[(row[0][0],'Growth rates'),'last relevant date']=last_relevant_sample
    hopkins_confirmed.at[(row[0][0],'Growth rates'),'GF_Q1']=lower
    hopkins_confirmed.at[(row[0][0],'Growth rates'),'GF_Q2']=med
    hopkins_confirmed.at[(row[0][0],'Growth rates'),'GF_Q3']=upper
    for cursor in range(len(actual_dates)):
        date=actual_dates[cursor]
        hopkins_confirmed.at[(row[0][0],'Growth rates'),date]=t_ratios[cursor]
    
      
        
hopkins_confirmed 
                 
#print ('ratios',ratios)
#print ('totals',totals)
##print ('lengths ', len(ratios),l) 
#print ('data', datarows[0][1])

#datarows [0][1]['1/29/2020'] 
    

first date is 1/22/2020 offset 15
rvrscur col 3/27/2020
dates Index(['1/22/2020', '1/23/2020', '1/24/2020', '1/25/2020', '1/26/2020',
       '1/27/2020', '1/28/2020', '1/29/2020', '1/30/2020', '1/31/2020',
       '2/1/2020', '2/2/2020', '2/3/2020', '2/4/2020', '2/5/2020', '2/6/2020',
       '2/7/2020', '2/8/2020', '2/9/2020', '2/10/2020', '2/11/2020',
       '2/12/2020', '2/13/2020', '2/14/2020', '2/15/2020', '2/16/2020',
       '2/17/2020', '2/18/2020', '2/19/2020', '2/20/2020', '2/21/2020',
       '2/22/2020', '2/23/2020', '2/24/2020', '2/25/2020', '2/26/2020',
       '2/27/2020', '2/28/2020', '2/29/2020', '3/1/2020', '3/2/2020',
       '3/3/2020', '3/4/2020', '3/5/2020', '3/6/2020', '3/7/2020', '3/8/2020',
       '3/9/2020', '3/10/2020', '3/11/2020', '3/12/2020', '3/13/2020',
       '3/14/2020', '3/15/2020', '3/16/2020', '3/17/2020', '3/18/2020',
       '3/19/2020', '3/20/2020', '3/21/2020', '3/22/2020', '3/23/2020',
       '3/24/2020', '3/25/2020', '3/26/2020', '3/27/2020'],
      

last 3/26/2020
((-34.9285, 138.6007), 'data') data
new South Australia Australia
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 9.0, 9.0, 16.0, 19.0, 20.0, 29.0, 29.0, 37.0, 42.0, 50.0, 67.0, 100.0, 134.0, 170.0, 170.0, 235.0, 257.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.6667, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, 0.2857, 0.0, 0.7778, 0.1875, 0.0526, 0.45, 0.0, 0.2759, 0.1351, 0.1905, 0.34, 0.4925, 0.34, 0.2687, 0.0, 0.3824, 0.0936]
NO SKIP
South Australia Australia [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.

last 3/26/2020
((-26.5225, 31.4659), 'data') data
new nan Eswatini
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 4.0, 4.0, 4.0, 6.0, 9.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.5, 0.5]
SKIP
((-23.4425, -58.4438), 'data') data
new nan Paraguay
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 5.0, 5.0, 6.0, 6.0, 6.0, 8.0, 9.0, 11.0, 11.0, 13.0, 18.0, 22.0, 22.0, 27.0, 37.0, 41.0, 52.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0,

last 3/26/2020
((-14.235, -51.9253), 'data') data
new nan Brazil
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 4.0, 4.0, 13.0, 13.0, 20.0, 25.0, 31.0, 38.0, 52.0, 151.0, 151.0, 162.0, 200.0, 321.0, 372.0, 621.0, 793.0, 1021.0, 1546.0, 1924.0, 2247.0, 2554.0, 2985.0, 3417.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.25, 0.0, 0.5385, 0.25, 0.24, 0.2258, 0.3684, 1.9038, 0.0, 0.0728, 0.2346, 0.605, 0.1589, 0.6694, 0.277, 0.2875, 0.5142, 0.2445, 0.1679, 0.1366, 0.1688, 0.1447]
NO SKIP
nan Brazil [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.25, 0.0, 0.5385, 0.25, 0.24, 0.2258, 0.3684, 1.9038, 0.0, 0.0728, 0.2346, 0.605, 0.1589, 0.6694, 0.277, 0.2875, 0.5142, 0.2445, 0.1679,

lower, median, upper 0.13335 0.2778 0.3188
last 3/26/2020
((-4.03829, 21.7587), 'data') data
new nan Congo (Brazzaville)
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.3333, 0.0, 0.0, 0.0, 0.0]
SKIP
((-1.9403, 29.8739), 'data') data
new nan Rwanda
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 5.0, 7.0, 8.0, 8.0, 17.0, 17.0, 19.0, 36.0, 40.0, 41.0, 50.0, 54.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

last 3/26/2020
((51.0, 9.0), 'data') data
new nan Germany
totals [0, 0, 0, 0, 0, 1.0, 4.0, 4.0, 4.0, 5.0, 8.0, 10.0, 12.0, 12.0, 12.0, 12.0, 13.0, 13.0, 14.0, 14.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 17.0, 27.0, 46.0, 48.0, 79.0, 130.0, 159.0, 196.0, 262.0, 482.0, 670.0, 799.0, 1040.0, 1176.0, 1457.0, 1908.0, 2078.0, 3675.0, 4585.0, 5795.0, 7272.0, 9257.0, 12327.0, 15320.0, 19848.0, 22213.0, 24873.0, 29056.0, 32986.0, 37323.0, 43938.0, 50871.0]
ratios [0, 0, 0, 0, 0, 4, 3.0, 0.0, 0.0, 0.25, 0.6, 0.25, 0.2, 0.0, 0.0, 0.0, 0.0833, 0.0, 0.0769, 0.0, 0.1429, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.5882, 0.7037, 0.0435, 0.6458, 0.6456, 0.2231, 0.2327, 0.3367, 0.8397, 0.39, 0.1925, 0.3016, 0.1308, 0.2389, 0.3095, 0.0891, 0.7685, 0.2476, 0.2639, 0.2549, 0.273, 0.3316, 0.2428, 0.2956, 0.1192, 0.1197, 0.1682, 0.1353, 0.1315, 0.1772, 0.1578]
NO SKIP
nan Germany [0, 0, 0, 0, 0, 4, 3.0, 0.0, 0.0, 0.25, 0.6, 0.25, 0

totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 3.0, 3.0, 3.0, 4.0, 6.0, 9.0, 24.0, 35.0, 35.0, 82.0, 102.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 2.0, 0.0, 0.0, 0.3333, 0.5, 0.5, 1.6667, 0.4583, 0.0, 1.3429, 0.2439]
NO SKIP
Newfoundland and Labrador Canada [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 2.0, 0.0, 0.0, 0.3333, 0.5, 0.5, 1.6667, 0.4583, 0.0, 1.3429, 0.2439] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 3.0, 3.0, 3.0, 4.0, 6.0, 9.0, 24.0, 35.0, 35.0, 82.0, 102.0]
lens 66

last 3/26/2020
((56.2639, 9.5018), 'data') data
new nan Denmark
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 3.0, 4.0, 4.0, 6.0, 10.0, 10.0, 23.0, 23.0, 35.0, 90.0, 262.0, 442.0, 615.0, 801.0, 827.0, 864.0, 914.0, 977.0, 1057.0, 1151.0, 1255.0, 1326.0, 1395.0, 1450.0, 1591.0, 1724.0, 1877.0, 2046.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 2.0, 0.3333, 0.0, 0.5, 0.6667, 0.0, 1.3, 0.0, 0.5217, 1.5714, 1.9111, 0.687, 0.3914, 0.3024, 0.0325, 0.0447, 0.0579, 0.0689, 0.0819, 0.0889, 0.0904, 0.0566, 0.052, 0.0394, 0.0972, 0.0836, 0.0887, 0.09]
NO SKIP
nan Denmark [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0, 2.0, 0.3333, 0.0, 0.5, 0.6667, 0.0, 1.3, 0.0, 0.5217, 1.5714, 1.9111, 0.687, 0.3914, 0.3024, 0.0325, 0.0447, 0.0579, 0.0689, 0.0819, 0.0889, 0.0904, 0.0566

new Yukon Canada
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.0, 3.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0]
SKIP
((64.8255, -124.8457), 'data') data
new Northwest Territories Canada
totals [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0]
ratios [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0.0]
SKIP
((64.9631, -19.0208), 'data') data
new nan Iceland
totals [0, 0, 0, 

Province_State  \
coordinate                     information                                  
(-41.4545, 145.9707)           data                              Tasmania   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-40.9006, 174.886)            data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-38.4161, -63.6167)           data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-37.8136, 144.9631)           data                              Victoria   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-35.6751, -71.543)            data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-35.4735, 149.0124)           data          Australian Capital Territory   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
...                                                                   ...   
(48.3794, 31.1656)             Growth rates                           NaN   
(48.669, 19.699)               Growth rates                           NaN   
(49.2827, -123.1207)           Growth rates                           NaN   
(49.3723, -2.3644)             Growth rates                           NaN   
(49.8153, 6.1296)              Growth rates                           NaN   
(49.8175, 15.472999999999999)  Growth rates                           NaN   
(50.8333, 4.0)                 Growth rates                           NaN   
(51.0, 9.0)                    Growth rates                           NaN   
(51.2538, -85.3232)            Growth rates                           NaN   
(51.9194, 19.1451)             Growth rates                           NaN   
(52.1326, 5.2913)              Growth rates                           NaN   
(52.9399, -106.4509)           Growth rates                           NaN   
(52.9399, -73.5491)            Growth rates                           NaN   
(53.1355, -57.6604)            Growth rates                           NaN   
(53.1424, -7.6921)             Growth rates                           NaN   
(53.7098, 27.9534)             Growth rates                           NaN   
(53.7609, -98.8139)            Growth rates                           NaN   
(53.9333, -116.5765)           Growth rates                           NaN   
(54.2361, -4.5481)             Growth rates                           NaN   
(55.1694, 23.8813)             Growth rates            

###### Calculate growth factors

In [119]:
hopkins_confirmed 
#hopkins_confirmed.columns



#dataonly=hopkins_confirmed['information']=='data'
#hopconf_data=hopkins_confirmed[dataonly]
#hopconf_data

Province_State  \
coordinate                     information                                  
(-41.4545, 145.9707)           data                              Tasmania   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-40.9006, 174.886)            data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-38.4161, -63.6167)           data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-37.8136, 144.9631)           data                              Victoria   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-35.6751, -71.543)            data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-35.4735, 149.0124)           data          Australian Capital Territory   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
...                                                                   ...   
(48.3794, 31.1656)             Growth rates                           NaN   
(48.669, 19.699)               Growth rates                           NaN   
(49.2827, -123.1207)           Growth rates                           NaN   
(49.3723, -2.3644)             Growth rates                           NaN   
(49.8153, 6.1296)              Growth rates                           NaN   
(49.8175, 15.472999999999999)  Growth rates                           NaN   
(50.8333, 4.0)                 Growth rates                           NaN   
(51.0, 9.0)                    Growth rates                           NaN   
(51.2538, -85.3232)            Growth rates                           NaN   
(51.9194, 19.1451)             Growth rates                           NaN   
(52.1326, 5.2913)              Growth rates                           NaN   
(52.9399, -106.4509)           Growth rates                           NaN   
(52.9399, -73.5491)            Growth rates                           NaN   
(53.1355, -57.6604)            Growth rates                           NaN   
(53.1424, -7.6921)             Growth rates                           NaN   
(53.7098, 27.9534)             Growth rates                           NaN   
(53.7609, -98.8139)            Growth rates                           NaN   
(53.9333, -116.5765)           Growth rates                           NaN   
(54.2361, -4.5481)             Growth rates                           NaN   
(55.1694, 23.8813)             Growth rates            

###### Save file as pickle

In [123]:
pickle_path = RESULT_PATH.format(datetime.now().strftime('%d%m'))
with open(pickle_path, 'wb') as file:
    pickle.dump(hopkins_confirmed, file)